<a href="https://colab.research.google.com/github/lsmanoel/Keras_DeepLearning_BreastCancerDiagnosis/blob/master/breast_cancer_diagnosis_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this exemple you will learn how to train a deep neural network to diagnose breast cancer. The packages required for this exemple are shown below:

In [0]:
import pandas as pd

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

To load the database is shown below. The input_data_breast are the predictors and the output_data_breast are de real results of breast cancer diagnoses for each predictor.

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

The database will be separated into two sections. A section to train and a section to evaluate the deep NN. The parameter test_size means the fraction value that will be used to evaluate the net.

In [0]:
predictors = pd.read_csv('gdrive/My Drive/Colab Notebooks/Keras_DeepLearning/binary_classifier/breast_cancer_diagnosis/entradas-breast.csv')
realResult = pd.read_csv('gdrive/My Drive/Colab Notebooks/Keras_DeepLearning/binary_classifier/breast_cancer_diagnosis/saidas-breast.csv')
predictors_training, predictors_test, realResult_training, realResult_test = train_test_split(predictors, realResult, test_size=0.10)


The structure of the Deep Neural Net is shown below. This net has only two hidden layers. The error function used is the Binary Crossentropy (ideal for binary problems). The output activation function is a sigmoid to provide a probability result (range between 0 and 1). Dropouts prevent network overfitting!

>1.   30 neurons in input layers 
>1.   8n in dense relu hidden  layers 
>1.   8n in dense relu hidden layer
>1.   1n in dense sigmoid output layer


The optimizer used is [Adam Optimizer](https://arxiv.org/abs/1412.6980v8).

Other types of optimizers:
[https://keras.io/optimizers/](https://keras.io/optimizers/)

In [0]:
def createNet():
  newNeuralClassifier = Sequential()
  
  newNeuralClassifier.add(Dense(units = 8, activation = 'relu',
                                kernel_initializer = 'random_uniform', 
                                input_dim = 30))
  newNeuralClassifier.add(Dropout(0.2))
  
  newNeuralClassifier.add(Dense(units = 8, 
                                activation = 'relu',
                                kernel_initializer = 'normal'))
  newNeuralClassifier.add(Dropout(0.2))
  
  newNeuralClassifier.add(Dense(units = 1, 
                                activation = 'sigmoid'))

  #lr: learning rate ... decay: decay of learning rate
  #clipvalue: limits the values of the weights
  optimizer_config = keras.optimizers.Adam(lr = 0.001, 
                                           decay = 0.0001, 
                                           clipvalue = 0.5)
  
  newNeuralClassifier.compile(optimizer = optimizer_config, 
                              loss = 'binary_crossentropy',
                              metrics = ['binary_accuracy'])
  return newNeuralClassifier

neuralClassifier = createNet()

To play the training:

In [0]:
neuralClassifier.fit(predictors_training, realResult_training,
                  batch_size = 10, epochs = 100)

To get and set the weights of layer or get config of the layer: [https://keras.io/layers/about-keras-layers/](https://keras.io/layers/about-keras-layers/)

In [0]:
weight0 = neuralClassifier.layers[0].get_weights()
weight1 = neuralClassifier.layers[1].get_weights()

config0 = neuralClassifier.layers[0].get_config()
config1 = neuralClassifier.layers[1].get_config()

reconstructed_layer = Dense.from_config(config0)
reconstructed_layer.set_weights(weight1)

To view the confusion matrix:

In [0]:
prediction = neuralClassifier.predict(predictors_test)
prediction = (prediction > 0.5)#Boolean convert

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(realResult_test, prediction)
matrix = confusion_matrix(realResult_test, prediction)

confusion_matrix_ = pd.DataFrame (matrix, 
                                  columns =['Predicted Positive', 'Predicted Negative'], 
                                  index =['Actual Positive', 'Actual Negative'])

confusion_matrix_



To evaluate the Net:

In [0]:
result = neuralClassifier.evaluate(predictors_test, realResult_test)
print("Error:%f Accuracy:%f"% (result[0], result[1]))

To save the neuralClassifier like json file:

In [0]:
neuralClassifier_json = neuralClassifier.to_json()
with open('neuralClassifier_breast_1.json', 'w') as json_file:
    json_file.write(neuralClassifier_json)

To save the weights:


In [0]:
neuralClassifier.save_weights('neuralClassifier_weights_breast_1.h5')


To open the saved neuralClassifier and weights:

In [0]:
import numpy as np
import pandas as pd
from keras.models import model_from_json

file = open('neuralClassifier_breast_1.json', 'r')
loadedNet = file.read()
file.close()

loadedClassifier = model_from_json(loadedNet)
loadedClassifier.load_weights('neuralClassifier_weights_breast_1.h5')

woman_data = np.array([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                        0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 
                        0.015, 0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                        0.84, 158, 0.363]])

diagnosis = loadedClassifier.predict(woman_data)
diagnosis = (diagnosis > 0.5)
print(diagnosis)


Below is shown how to perform a cross-validation of a Keras Classifier with scikit_learn. The parameter cv = 10 indicates that the database will be divided into 10 pieces to execute the cross validation.

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

crossClassifier = KerasClassifier(build_fn = createNet,
                                  epochs = 150,
                                  batch_size =50)

crossResult = cross_val_score(estimator = crossClassifier,
                              X = predictors, y = realResult,
                              cv = 10, scoring = 'accuracy')

resultMean = crossResult.mean()
resultDeviation = crossResult.std()

print("Mean:%f Standard Deviation:%f"% (resultMean, resultDeviation))

To find the best combination of parameters we may use the GridSearchCV. This function will find the best combination of the values  passed in the dict array testeParameters. This process may take a long time to execute (~8 hours)... wait sit (or sleeping).

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


def createNet_tunning(optimizer, lossFunc, kernel_initializer, activation, neurons1, neurons2):
    newNeuralClassifier = Sequential()
    
    newNeuralClassifier.add(Dense(units = neurons1, 
                                  activation = activation, 
                                  kernel_initializer = kernel_initializer, input_dim = 30))
    newNeuralClassifier.add(Dropout(0.2))
    
    newNeuralClassifier.add(Dense(units = neurons2, 
                                  activation = activation, 
                                  kernel_initializer = kernel_initializer))
    newNeuralClassifier.add(Dropout(0.2))
    
    newNeuralClassifier.add(Dense(units = 1, activation = 'sigmoid'))
    
    newNeuralClassifier.compile(optimizer = optimizer, 
                                loss = lossFunc,
                                metrics = ['binary_accuracy'])
    
    return newNeuralClassifier

tunningClassifier = KerasClassifier(build_fn = createNet_tunning)

testParameters = {'batch_size': [10, 30],
                  'epochs': [50, 100],
                  'optimizer': ['adam', 'sgd'],
                  'lossFunc': ['binary_crossentropy', 'hinge'],
                  'kernel_initializer': ['random_uniform', 'normal'],
                  'activation': ['relu', 'tanh'],
                  'neurons1': [16, 8],
                  'neurons2': [16, 8, 4]}

grid_search = GridSearchCV(estimator = tunningClassifier,
                           param_grid = testParameters,
                           scoring = 'accuracy',
                           cv = 5)

grid_search = grid_search.fit(predictors, realResult)
bestParameters = grid_search.best_params_
bestAccuracy = grid_search.best_score_

print(bestParameters)
print(bestAccuracy)